In [ ]:
import clr
import sys, os
from bonsai_pylib import load_bonsai_config

load_bonsai_config()

from System import Byte, Array
import System

clr.AddReference("Bonsai.Harp")
import Bonsai.Harp as harp



In [ ]:
a = harp.HarpMessage.FromSingle(32, harp.MessageType.Write, 1.0)
print(harp.HarpMessage.get_Address(a))
print(harp.HarpMessage.get_PayloadType(a))

a = harp.HarpMessage.FromSingle(32, harp.MessageType.Write, 1.0)
print(harp.HarpMessage.get_Address(a))
print(harp.HarpMessage.get_PayloadType(a))
print(harp.HarpMessage.GetPayloadSingle(a))


In [ ]:
a = harp.HarpMessage.FromByte(32, harp.MessageType.Write, 20)
print(harp.HarpMessage.get_Address(a))
print(harp.HarpMessage.get_PayloadType(a))
print(harp.HarpMessage.GetPayloadByte(a))

a.Address

In [ ]:
## Read Async

device = harp.AsyncDevice("COM5")
print(device.ReadHardwareVersionAsync().get_Result())
print(device.ReadByteAsync(11).get_Result())
print(device.ReadInt16Async(44).get_Result())
print(device.WriteByteAsync(11,1).get_Result())
device.Dispose()
